In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from IPython.display import display

print("📌 가설2: 디커플링 → 신용경색 Granger 시작")

# ===============================
# 1️⃣ 디커플링 로드 + 월 평균 처리
# ===============================
df_dec = pd.read_csv("../../윤성/data/decoupling.csv")

df_dec["date"] = pd.to_datetime(df_dec["date"])
df_dec = df_dec.groupby("date", as_index=False)["decoupling"].mean()
df_dec = df_dec.sort_values("date").reset_index(drop=True)

print("디커플링 데이터 확인")
display(df_dec.head())

# ===============================
# 2️⃣ 신용스프레드 로드
# ===============================
df_sp = pd.read_csv("../../윤성/data/preprocessed/전처리_신용스프레드.csv")

# 날짜 컬럼 자동 탐색
date_col = [c for c in df_sp.columns if "date" in c.lower() or "기간" in c][0]
df_sp = df_sp.rename(columns={date_col:"date"})
df_sp["date"] = pd.to_datetime(df_sp["date"])

# 스프레드 컬럼 자동 탐색
val_col = [c for c in df_sp.columns if c!="date"][0]
df_sp = df_sp[["date", val_col]].rename(columns={val_col:"SPREAD"})
df_sp = df_sp.sort_values("date").reset_index(drop=True)

print("스프레드 데이터 확인")
display(df_sp.head())

# ===============================
# 3️⃣ 병합 (월 기준 유지)
# ===============================
df = df_dec.merge(df_sp, on="date", how="inner")
df = df.sort_values("date").reset_index(drop=True)

print("병합 결과")
display(df.head())

# ===============================
# 4️⃣ 정상성 확보 (1차 차분)
# ===============================
df["DECOUPLING_d1"] = df["decoupling"].diff()
df["SPREAD_d1"] = df["SPREAD"].diff()

df = df.dropna().reset_index(drop=True)

print("차분 후 데이터")
display(df.head())

# ===============================
# 5️⃣ Granger 인과검정
# ===============================
def run_granger(df, cause, effect, maxlag=6):
    data = df[[effect, cause]].dropna()
    print("\n" + "="*60)
    print(f"{cause} → {effect} (maxlag={maxlag})")
    print("="*60)

    res = grangercausalitytests(data, maxlag=maxlag, verbose=False)
    pvals = {lag: res[lag][0]["ssr_ftest"][1] for lag in res.keys()}

    out = pd.DataFrame({
        "lag": list(pvals.keys()),
        "p_value": list(pvals.values())
    }).sort_values("lag")

    display(out)

    best_lag = out.loc[out["p_value"].idxmin(),"lag"]
    best_p = out["p_value"].min()

    print(f"최소 p-value: {best_p:.4f} (lag{int(best_lag)})")

print("✅ 디커플링 → 스프레드")
run_granger(df, "DECOUPLING_d1", "SPREAD_d1", maxlag=6)

print("\n🎯 가설2 Granger 완료")


📌 가설2: 디커플링 → 신용경색 Granger 시작
디커플링 데이터 확인


,date,decoupling
0,2004-01-01,2.011135
1,2004-02-01,1.620480
2,2004-03-01,1.882683
3,2004-04-01,1.393422
4,2004-05-01,1.266506


스프레드 데이터 확인


,date,SPREAD
0,2004-01-01,0.79
1,2004-02-01,0.80
2,2004-03-01,0.79
3,2004-04-01,0.75
4,2004-05-01,0.68


병합 결과


...

,date,decoupling,SPREAD,DECOUPLING_d1,SPREAD_d1
0,2004-02-01,1.620480,0.80,-0.390655,0.01
1,2004-03-01,1.882683,0.79,0.262203,-0.01
2,2004-04-01,1.393422,0.75,-0.489261,-0.04
3,2004-05-01,1.266506,0.68,-0.126916,-0.07
4,2004-06-01,1.011087,0.63,-0.255419,-0.05


✅ 디커플링 → 스프레드

DECOUPLING_d1 → SPREAD_d1 (maxlag=6)


c:\anaconda\envs\data\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


,lag,p_value
0,1,0.001848
1,2,0.004926
2,3,0.011862
3,4,0.017186
4,5,0.037417
5,6,0.086710


최소 p-value: 0.0018 (lag1)

🎯 가설2 Granger 완료


## 📌 가설 2: 디커플링 → 신용경색 신호 (Granger 인과검정 결과)

### 1️⃣ 분석 목적

본 분석은 디커플링 지수의 변화가 이후 신용스프레드 확대를
선행적으로 설명하는지를 검정하기 위해 수행되었다.

즉,

> 디커플링 지수가 상승하면 이후 신용스프레드가 확대되는가?

를 Granger 인과검정을 통해 확인하였다.

---

### 2️⃣ 분석 방법

- 데이터: 월 단위
- 사용 변수:
  - DECOUPLING_d1 (디커플링 지수 1차 차분)
  - SPREAD_d1 (AA- 회사채 스프레드 1차 차분)
- 최대 시차: 6개월
- 검정 방법: Granger Causality Test (ssr F-test 기반)

---

### 3️⃣ 검정 결과

| lag | p-value |
|-----|----------|
| 1 | 0.0018 |
| 2 | 0.0049 |
| 3 | 0.0119 |
| 4 | 0.0172 |
| 5 | 0.0374 |
| 6 | 0.0867 |

최소 p-value: **0.0018 (lag 1)**

---

### 4️⃣ 통계적 판단

유의수준 5% 기준에서

- lag 1 ~ lag 5 모두 p < 0.05
- 특히 lag 1에서 매우 강한 유의성 (p=0.0018)

따라서,

> 디커플링 지수의 변화는 이후 신용스프레드 변화를 Granger-cause 한다.

즉, 귀무가설(H0):

> 디커플링은 신용스프레드를 예측하지 못한다

는 **기각된다.**

---

### 5️⃣ 경제적 해석

이 결과는 다음을 의미한다.

- 금융–실물 괴리가 확대될 경우
- 약 1~5개월 이후
- 신용스프레드가 유의하게 확대되는 경향이 있다.

이는 다음의 논리와 일치한다:

> 실물 둔화가 즉시 신용위험에 반영되기보다는  
> 일정 시차를 두고 기업 부도위험이 재평가되며  
> 신용 프리미엄이 상승한다.

즉, 디커플링은 단순한 괴리 지표가 아니라

> **신용경색의 선행 신호(early warning signal)** 로 기능할 가능성이 높다.

---

### 6️⃣ 1번 가설과의 연결

1번 가설에서 금융–실물 괴리 구조를 확인하였다면,

2번 가설에서는 그 괴리가

> 실제 금융시장 위험지표(신용스프레드)로 전이되는 동태적 경로

를 통계적으로 입증한 것이다.

즉,

금융–실물 괴리 → 신용위험 확대

라는 전이 구조가 실증적으로 확인되었다.

---

## 🎯 최종 결론

디커플링 지수의 증가는 통계적으로 유의하게
이후 신용스프레드 확대를 선행한다.

따라서,

> 디커플링은 신용경색의 조기 경고 신호로 활용될 수 있다.
